## **1. Запустим первую версию бота**

Скачиваем библиотеки

In [ ]:
!pip3 install pytelegrambotapi

In [ ]:
import telebot
# Здесь нужно вставить токен, который дал BotFather при регистрации
# Пример: token = '5880107702:AAGKicINDRKjNaQVooWtlZT1uqXvnRd9U'
token = '5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U'  # <<< Ваш токен

# В этой строчке мы заводим бота и даем ему запомнить токен
bot = telebot.TeleBot(token)

# Пишем первую функцию, которая отвечает "Привет" на команду /start
# Все функции общения приложения с ТГ спрятаны в функции под @
@bot.message_handler(commands=['start'])
def say_hi(message):
    # Функция, отправляющая "Привет" в ответ на команду /start
    bot.send_message(message.chat.id, 'Привет')

# Запускаем бота. Он будет работать до тех пор, пока работает ячейка (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()

## **2. Бот отвечает на вопросы**



In [ ]:
# Замените 'YOUR_BOT_TOKEN' на токен вашего Telegram-бота
bot = telebot.TeleBot('5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U')

# Обработчик для команды /start
@bot.message_handler(commands=['start'])
def handle_start(message):
    bot.send_message(message.chat.id, 'Привет! Я ваш бот. Чем я могу вам помочь?')

# Обработчик для получения текстовых сообщений
@bot.message_handler(func=lambda message: True)
def handle_text(message):
    user_message = message.text
    response = generate_response(user_message)
    bot.send_message(message.chat.id, response)

# Генерация ответа на вопрос
def generate_response(user_message):
    user_message = user_message.lower()

    if 'как дела' in user_message:
        return 'У меня всё хорошо, спасибо! А как у вас?'

    if 'погода' in user_message:
        return 'Сегодня на улице пасмурно'

    if 'что делаешь' in user_message:
        return 'Я отвечаю на ваши вопросы. Задайте что-нибудь интересное!'

    return 'Извините, я не понял ваш вопрос.'

# Запускаем бота
bot.polling()

## **3. Бот - Энциклопедия**

In [ ]:
!pip3 install wikipedia

In [ ]:
import telebot, wikipedia, re
# Создаем экземпляр бота
bot = telebot.TeleBot('5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U')
# Выбираем русский язык в Wikipedia
wikipedia.set_lang("ru")
# Чистим текст статьи из Wikipedia и ограничиваем его тысячей символов
def getwiki(s):
    try:
        ny = wikipedia.page(s)
        # Получаем первую тысячу символов
        wikitext=ny.content[:1000]
        # Разбиваем, считая точку разделителем
        wikimas=wikitext.split('.')
        # Отбрасываем всё после последней точки
        wikimas = wikimas[:-1]
        # Создаем пустую переменную для текста
        wikitext2 = ''
        # Проходимся по строкам, где нет знаков «равно» (то есть все, кроме заголовков)
        for x in wikimas:
            if not('==' in x):
                    # Если в строке осталось больше трех символов, добавляем ее
                    # к нашей переменной и возвращаем утерянные при разделении строк точки на место
                if(len((x.strip()))>3):
                   wikitext2=wikitext2+x+'.'
            else:
                break
        # Теперь при помощи регулярных выражений убираем разметку
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\([^()]*\)', '', wikitext2)
        wikitext2=re.sub('\{[^\{\}]*\}', '', wikitext2)
        # Возвращаем текстовую строку
        return wikitext2
    # Обрабатываем исключение, которое мог вернуть модуль wikipedia при запросе
    except Exception as e:
        return 'В энциклопедии нет информации об этом'
# Функция, обрабатывающая команду /start
@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Введите вопрос')
# Получение сообщений от пользователя
@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, getwiki(message.text))
# Запускаем бота
bot.polling()

## **4. Бот для перевода голоса в текст** (один язык)

In [ ]:
!pip install pyTelegramBotAPI SpeechRecognition pydub

In [ ]:
import os
import telebot
import speech_recognition
from pydub import AudioSegment

In [ ]:
# ↓↓↓ Ниже нужно вставить токен, который дал BotFather при регистрации
# Пример: token = '2007628239:AAEF4ZVqLiRKG7j49EC4vaRwXjDN6xng8'
token = '5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U'

bot = telebot.TeleBot(token)

def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_') # Если в имени файла есть символ "/", заменить на символ "_".
    with open(filename, 'wb') as f: # Файл создается и открывается для записи в бинарном режиме
        f.write(downloaded_file) # Содержимое скачанного файла (downloaded_file) записывается в созданный файл с именем filename
    return filename

def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename


def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    return text

@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая текст в ответ на голосовое
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    bot.send_message(message.chat.id, text)


# Запускаем бота. Он будет работать до тех пор, пока
# работает ячейка (крутится значок слева).
# Остановим ячейку - остановится бот
bot.polling()

## **5. Бот для перевода голоса в текст** (с русского на английский)

In [ ]:
!pip install translate

In [ ]:
import telebot
import os
import speech_recognition
from pydub import AudioSegment
from translate import Translator  # Используем библиотеку translate

token = '5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U'
bot = telebot.TeleBot(token)

def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename

def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename

def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()

    with speech_recognition.WavFile(wav_filename) as source:
        wav_audio = recognizer.record(source)

    text = recognizer.recognize_google(wav_audio, language='ru')

    if os.path.exists(oga_filename):
        os.remove(oga_filename)

    if os.path.exists(wav_filename):
        os.remove(wav_filename)

    return text

def translate_text(text):
    # Перевести текст с русского на английский
    translator = Translator(to_lang='en', from_lang='ru')
    translated_text = translator.translate(text)
    return translated_text

@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая текст в ответ на голосовое
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    translated_text = translate_text(text)  # Переводим текст с русского на английский
    bot.send_message(message.chat.id, translated_text)

bot.polling()

## **6. Бот для перевода голоса в голос** (с русского на английский)

In [ ]:
!pip install gTTS
!pip install pydub
!pip install translate
!pip install SpeechRecognition

In [ ]:
import telebot
import os
import speech_recognition
from pydub import AudioSegment
from translate import Translator
from gtts import gTTS

token = '5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U'
bot = telebot.TeleBot(token)

def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename

def oga2wav(filename):
    # Конвертация формата файлов
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_file(filename)
    audio.export(new_filename, format='wav')
    return new_filename


def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.AudioFile(wav_filename) as source:
        wav_audio = recognizer.record(source)
    text = recognizer.recognize_google(wav_audio, language='ru')
    if os.path.exists(oga_filename):
        os.remove(oga_filename)
    if os.path.exists(wav_filename):
        os.remove(wav_filename)
    return text

def translate_text(text):
    # Перевести текст с русского на английский
    translator = Translator(to_lang='en', from_lang='ru')
    translated_text = translator.translate(text)
    return translated_text

@bot.message_handler(content_types=['voice'])
def transcript(message):
    # Функция, отправляющая голосовое сообщение на английском
    filename = download_file(bot, message.voice.file_id)
    text = recognize_speech(filename)
    translated_text = translate_text(text)  # Переводим текст с русского на английский

    # Создаем голосовое сообщение на английском
    tts = gTTS(translated_text, lang='en')
    tts.save('translated_voice.mp3')

    # Отправляем голосовое сообщение
    with open('translated_voice.mp3', 'rb') as audio:
        bot.send_voice(message.chat.id, audio)

    # Удаляем временные файлы
    if os.path.exists(filename):
        os.remove(filename)
    if os.path.exists('translated_voice.mp3'):
        os.remove('translated_voice.mp3')

bot.polling()

## **7. Бот для перевода голоса в голос** (с русского на английский/немецкий)

In [ ]:
!pip install pydub
!pip install SpeechRecognition
!pip install gTTS
!pip install translate

In [ ]:
import telebot
import os
import speech_recognition
from pydub import AudioSegment
from translate import Translator
from gtts import gTTS
from telebot import types

token = '5880107702:AAGKicINDRfRKjNaQVooWtlZT1uqXvnRd9U'
bot = telebot.TeleBot(token)

def download_file(bot, file_id):
    # Скачивание файла, который прислал пользователь
    file_info = bot.get_file(file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    filename = file_id + file_info.file_path
    filename = filename.replace('/', '_')
    with open(filename, 'wb') as f:
        f.write(downloaded_file)
    return filename

def oga2wav(filename):
    # Конвертация формата файла из .oga в .wav
    new_filename = filename.replace('.oga', '.wav')
    audio = AudioSegment.from_ogg(filename)
    audio.export(new_filename, format='wav')
    return new_filename

def recognize_speech(oga_filename):
    # Перевод голоса в текст + удаление использованных файлов
    wav_filename = oga2wav(oga_filename)
    recognizer = speech_recognition.Recognizer()
    with speech_recognition.AudioFile(wav_filename) as source:
        wav_audio = recognizer.record(source)
    text = recognizer.recognize_google(wav_audio, language='ru')
    if os.path.exists(oga_filename):
        os.remove(oga_filename)
    if os.path.exists(wav_filename):
        os.remove(wav_filename)
    return text

def translate_text(text, target_language):
    # Перевести текст на выбранный язык
    translator = Translator(to_lang=target_language, from_lang='ru')
    translated_text = translator.translate(text)
    return translated_text

user_language = {}

@bot.message_handler(commands=['start'])
def start(message):
    # Функция, начинающая диалог с пользователем
    user_language[message.chat.id] = None  # Сбрасываем язык при старте
    ask_for_translation(message)

def ask_for_translation(message):
    # Функция, запрашивающая у пользователя язык перевода
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    item_en = types.KeyboardButton('Английский')
    item_de = types.KeyboardButton('Немецкий')
    markup.add(item_en, item_de)
    bot.send_message(message.chat.id, 'На какой язык перевести?', reply_markup=markup)
    bot.register_next_step_handler(message, get_user_language)

def get_user_language(message):
    # Функция для получения выбранного пользователем языка перевода
    if message.text.lower() == 'английский':
        user_language[message.chat.id] = 'en'
    elif message.text.lower() == 'немецкий':
        user_language[message.chat.id] = 'de'
    else:
        bot.send_message(message.chat.id, 'Пожалуйста, выберите язык из списка.')
        ask_for_translation(message)

    bot.send_message(message.chat.id, 'Пожалуйста, отправьте голосовое сообщение.')

@bot.message_handler(content_types=['voice'])
def translate_voice(message):
    # Функция, которая переводит голосовое сообщение
    if user_language[message.chat.id] is None:
        bot.send_message(message.chat.id, 'Пожалуйста, начните с команды /start и выберите язык перевода.')
        return

    target_language = user_language[message.chat.id]

    if hasattr(message.voice, 'file_id'):
        filename = download_file(bot, message.voice.file_id)
        text = recognize_speech(filename)
        translated_text = translate_text(text, target_language)

        tts = gTTS(translated_text, lang=target_language)
        tts.save('translated_voice.mp3')

        with open('translated_voice.mp3', 'rb') as audio:
            bot.send_voice(message.chat.id, audio)

        if os.path.exists(filename):
            os.remove(filename)
        if os.path.exists('translated_voice.mp3'):
            os.remove('translated_voice.mp3')
    else:
        bot.send_message(message.chat.id, 'Пожалуйста, отправьте голосовое сообщение.')

bot.polling()